# Baseline accuracy - measuring EPA predictions against EPA observations

In [1]:
#import libraries
import pandas as pd
import numpy as np
import regex as re
import datetime
import timeit

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import (roc_auc_score, confusion_matrix, accuracy_score, roc_curve, auc, 
classification_report, hamming_loss, mean_absolute_error)

import geopandas as gpd
from shapely.geometry import Point

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Load in data

In [2]:
def get_data(filename):
    #print(type(filename), filename)
    
    #1. read in data
    data = pd.read_csv(filename)
    #print(data.shape)
    
    #2. drop null index col, null rows where date does not exist in month
    data.drop(['Unnamed: 0'], axis=1, inplace = True)
    data = data[data['state code'].isnull() == False]
    #print(data.shape)
    
    #3. convert data columns to desired type
    data['issue date'] = pd.to_datetime(data['issue date'])
    data['valid date'] = pd.to_datetime(data['valid date'])
    
    data['record sequence'] = data['record sequence'].astype('int64')
    data['action day']=[True if day == 'Yes' else False for day in data['action day']]    #no missing values
    
    data['urldate'] = pd.to_datetime(data['urldate'])
    #print(data.dtypes)
    
    #4. create calculated data column: feature
    data['categorical'] = [1 if (cat == 'Good' or cat == 'Moderate') else 0 for cat in data['AQI category']]
    #print(data.shape)
    
    return(data)

In [452]:
data = get_data('data.csv')
data.shape

(1897901, 19)

In [13]:
#modifying for datatype F = forecast or O = day of observation datatype
    #compare EPA-predicted AQI category against actual AQI category

def relevant_data(datadf, datatype, state = 'CA'):
    datadf = datadf[datadf['data type'] == datatype]
    datadf = datadf[datadf['state code'] == state]
    datadf.drop_duplicates(['valid date', 'reporting area'], keep = 'first', inplace = True)
    return (datadf)

In [15]:
def distance(datadf, state = 'CA', reparea = 'San Francisco'):
    
    #dropping state-specific reporting areas with null lat / long in CA
    if state == 'CA':
        #print(datadf.shape)
        datadf = datadf[datadf['reporting area'] != 'TestA']
        datadf = datadf[datadf['reporting area'] != 'TestC']
        #print(datadf.shape)

    #set up gdf
    geometry = [Point(xy) for xy in zip(datadf['longitude'], datadf['latitude'])]
    #print(len(geometry))

    crs = {'init' :'epsg:4326'}
    gdf = gpd.GeoDataFrame(datadf, crs = crs, geometry = geometry)
    #print(gdf.shape)
    #display(gdf.head(5))
    
    #set up distance
    gdf['desiredarea_geometry'] = gdf[gdf['reporting area'] == reparea]['geometry'].values[0]
    desiredarea_geometry = gdf[gdf['reporting area'] == reparea]['geometry'].values[0]
    
    distance = gdf['geometry'].distance(desiredarea_geometry)
    gdf['distance'] = distance
    datadf['distance'] = distance    #set in both geopandas and pandas dfs
    #display(datadf.head(5))
    
    return(datadf)

In [16]:
def predictors(n, datadf, reparea = 'San Francisco'):
    #modified from Geo-spatial nb: sort by unique reporting area by distance
        #maintain original datadf values
    predictor_areas = datadf.drop_duplicates(['reporting area'], keep = 'first', inplace = False)    
    #print(predictor_areas.shape)
    
    predictor_areas = predictor_areas.sort_values(['distance']).head(n+1)    
    predictor_areas = predictor_areas[predictor_areas['reporting area'] != reparea]
    #print(predictor_areas.shape)

    #assign inverse weight based on distance in predictor df
    weight = [i/sum(predictor_areas['distance']) for i in predictor_areas['distance']]
    predictor_areas['weight'] = weight[::-1]
    
    return (predictor_areas)

In [61]:
def aqicat(df):
    aqicats = []
    for cat in df['AQI category']:
        #print(cat)
        if cat == 'Good':
            aqicats.append(0)
        elif cat == 'Moderate':
            aqicats.append(50)
        elif cat == 'Unhealthy for Sensitive Groups':
            aqicats.append(100)
        elif cat == 'Unhealthy':
            aqicats.append(150)
        elif cat == 'Very Unhealthy':
            aqicats.append(200)
        elif cat == 'Hazardous':
            aqicats.append(300)
        else:
            aqicats.append(404)
    return (aqicats)

# Forecasted data

In [471]:
#incorporate functions to return forecasted data
def forecasted(datadf, n, state = 'CA', reparea = 'San Francisco'):
    statedata_f = relevant_data(datadf, 'F', state = state)
    #print(statedata_f.shape)

    statedata_f = distance(statedata_f, state = state, reparea = reparea)
    #print(statedata_f.shape)

    pred = predictors(n, statedata_f)
    #display(pred)
    
    areas = list(pred['reporting area'])
    areas.append(reparea)                                                       #FILTER for incl / excl target reparea
    statedata_f = statedata_f[statedata_f['reporting area'].isin(areas)]    
    print(areas)
    print(statedata_f.shape)
    print(statedata_f['AQI category'].unique())
    
    statedata_f['aqicat'] = aqicat(statedata_f)
    
    return (statedata_f)

In [445]:
cadata_f3 = forecasted(data, 3)

['Oakland', 'San Rafael', 'Redwood City', 'San Francisco']
(1560, 21)
['Good' 'Moderate' 'Unhealthy for Sensitive Groups']


# Observed data

In [472]:
#incorporate functions to return observed data
def observed(datadf, n, state = 'CA', reparea = 'San Francisco'):
    statedata_o = relevant_data(datadf, 'O', state = state)
    #print(statedata_o.shape)

    statedata_o = distance(statedata_o, state = state, reparea = reparea)
    #print(statedata_o.shape)

    pred = predictors(n, statedata_o)
    #display(pred)

    areas = list(pred['reporting area'])
    areas.append(reparea)                                                       #FILTER for incl / excl target reparea
    statedata_o = statedata_o[statedata_o['reporting area'].isin(areas)]
    print(areas)
    print(statedata_o.shape)
    print(statedata_o['AQI category'].unique())
    
    statedata_o['aqicat'] = aqicat(statedata_o)
    
    return (statedata_o)

In [65]:
cadata_o3 = observed(data, 3)

['Oakland', 'San Rafael', 'Redwood City']
(1080, 21)
['Moderate' 'Good' 'Unhealthy for Sensitive Groups' 'Unhealthy']


In [66]:
cadata_f3.head()

,issue date,valid date,valid time,time zone,record sequence,data type,primary,reporting area,state code,latitude,...,AQI value,AQI category,action day,discussion,forecast source,urldate,categorical,geometry,distance,aqicat
3142,2017-06-01,2017-06-01,NaN,PDT,0,F,Y,Oakland,CA,37.8,...,36.0,Good,False,NaN,San Francisco Bay Area AQMD,1970-01-01 00:00:00.020170601,1,POINT (-122.27 37.8),0.167631,0
3143,2017-06-01,2017-06-02,NaN,PDT,1,F,Y,Oakland,CA,37.8,...,41.0,Good,False,NaN,San Francisco Bay Area AQMD,1970-01-01 00:00:00.020170601,1,POINT (-122.27 37.8),0.167631,0
3144,2017-06-01,2017-06-03,NaN,PDT,2,F,Y,Oakland,CA,37.8,...,NaN,Good,False,NaN,San Francisco Bay Area AQMD,1970-01-01 00:00:00.020170601,1,POINT (-122.27 37.8),0.167631,0
3145,2017-06-01,2017-06-04,NaN,PDT,3,F,Y,Oakland,CA,37.8,...,NaN,Good,False,NaN,San Francisco Bay Area AQMD,1970-01-01 00:00:00.020170601,1,POINT (-122.27 37.8),0.167631,0
3146,2017-06-01,2017-06-05,NaN,PDT,4,F,Y,Oakland,CA,37.8,...,NaN,Good,False,NaN,San Francisco Bay Area AQMD,1970-01-01 00:00:00.020170601,1,POINT (-122.27 37.8),0.167631,0


In [82]:
#return AQI category (dummied) per date per unique reporting area
#mean here returns actual value since no duplicate rows per date
cadata_f3.groupby(['valid date', 'reporting area'])['aqicat'].mean().head(10)

valid date  reporting area
2017-06-01  Oakland           0
            Redwood City      0
            San Rafael        0
2017-06-02  Oakland           0
            Redwood City      0
            San Rafael        0
2017-06-03  Oakland           0
            Redwood City      0
            San Rafael        0
2017-06-04  Oakland           0
Name: aqicat, dtype: int64

In [83]:
cadata_o3.groupby(['valid date', 'reporting area'])['aqicat'].mean().head(10)

valid date  reporting area
2017-06-05  Oakland           50
            Redwood City       0
            San Rafael        50
2017-06-06  Oakland           50
            Redwood City       0
            San Rafael        50
2017-06-07  Oakland            0
            Redwood City       0
            San Rafael         0
2017-06-08  Oakland            0
Name: aqicat, dtype: int64

In [103]:
#joining together so no dates are lost - will use df valid date column as index
cadata3 = pd.concat([cadata_f3, cadata_o3], axis = 1)
cadata3.shape

(2250, 44)

In [179]:
def compare(datadf):
    
    all_aqis = {}
    for area in datadf['reporting area'].unique():
        #print(area)
        
        area_aqis = {}
        missing_aqi = 0
        
        for date in datadf['valid date'].unique():
            
            if datadf[(datadf['reporting area'] == area) & (datadf['valid date'] == date)]['AQI category'].any():
                areadate_aqi = datadf[(datadf['reporting area'] == area) & (datadf['valid date'] == date)]['AQI category'].values[0]
                
                #date formatting
                date = str(date)[0:10]
                area_aqis[date] = areadate_aqi
                #print(date, areadate_aqi)
            
            else:
                areadate_aqi = 0
                missing_aqi += 1
                
                #date formatting
                date = str(date)[0:10]
                area_aqis[date] = areadate_aqi
                #print(date, areadate_aqi)
        
        print('Number of Dates missing aqi in '+str(area)+':', missing_aqi)
        all_aqis[area] = area_aqis
    
    return (all_aqis)
    

In [180]:
aqis_f3 = compare(cadata_f3)

print(aqis_f3.keys())
print(aqis_f3['Oakland']['2017-06-01'])

Number of Dates missing aqi in Oakland: 0
Number of Dates missing aqi in Redwood City: 0
Number of Dates missing aqi in San Rafael: 0
dict_keys(['Oakland', 'Redwood City', 'San Rafael'])
Good


In [181]:
aqis_o3 = compare(cadata_o3)

Number of Dates missing aqi in Oakland: 2
Number of Dates missing aqi in Redwood City: 20
Number of Dates missing aqi in San Rafael: 8


In [182]:
print(aqis_o3.keys())
print(aqis_o3['Oakland']['2017-06-10'])

dict_keys(['Oakland', 'Redwood City', 'San Rafael'])
Good


In [200]:
f3 = pd.DataFrame(aqis_f3)
o3 = pd.DataFrame(aqis_o3)

#list of dfs
dfs = {'For': f, 'Obs': o}
keys = ['{}'.format(df) for df in list(dfs.keys())]

results3 = pd.concat([f, o], axis = 1, keys = keys)

In [212]:
results3.head(15)

For                               Obs                        
             Oakland Redwood City San Rafael   Oakland Redwood City San Rafael
2017-06-01      Good         Good       Good       NaN          NaN        NaN
2017-06-02      Good         Good       Good       NaN          NaN        NaN
2017-06-03      Good         Good       Good       NaN          NaN        NaN
2017-06-04      Good         Good       Good       NaN          NaN        NaN
2017-06-05      Good         Good       Good  Moderate         Good   Moderate
2017-06-06      Good         Good       Good  Moderate         Good   Moderate
2017-06-07      Good         Good       Good      Good         Good       Good
2017-06-08      Good         Good       Good      Good         Good       Good
2017-06-09      Good         Good       Good      Good         Good       Good
2017-06-10      Good         Good       Good      Good         Good       Good
2017-06-11      Good         Good       Good      Good         Good       Good
2017-06-12      Good         Good       Good  Moderate         Good       Good
2017-06-13      Good         Good       Good  Moderate         Good       Good
2017-06-14  Moderate         Good       Good  Moderate         Good   Moderate
2017-06-15  Moderate         Good       Good      Good         Good       Good

In [467]:
#calculating baseline accuracy

def baseline(resultsdf):
    
    #set up dictionary for results per area
    results = {}
    for area in resultsdf['For'].columns:
        print(area)
        
        correct = 0
        incorrect = 0
        missing_all = 0
        missing_for = 0
        missing_obs = 0
        
        for date in resultsdf.index:
            for_aqi = str(resultsdf['For'][area].loc[date])
            obs_aqi = str(resultsdf['Obs'][area].loc[date])
            #print(date, type(for_aqi), for_aqi, type(obs_aqi), obs_aqi)
            
            if (for_aqi == 'nan') and (obs_aqi == 'nan'):
                missing_all += 1
                
            elif (for_aqi == 'nan') and (obs_aqi != 'nan'):
                missing_for += 1
                
            elif (for_aqi != 'nan') and (obs_aqi == 'nan'):
                missing_obs += 1
            
            elif for_aqi == obs_aqi:
                correct += 1
            
            elif for_aqi != obs_aqi:
                incorrect += 1
            else:
                print(area, date, 'Error')
        
        metrics = [correct, incorrect, missing_all, missing_for, missing_obs]
        results[area] = metrics
    
    #print(results)
    
    #calculate percentages
    for area in results.keys():
        total = sum(results[area])
        #print(area, total, results[area])
        
        percents = []
        for elem in results[area]:
            percent = np.round(elem/total, 4)
            percents.append(percent)
            #print(percent)
        
        #print('\n')
        results[area] = percents
    
    print(total)
    metrics = str('[correct, incorrect, missing_all, missing_for, missing_obs]')
    #print(metrics)
    return(metrics[1:len(metrics)-1], results)

In [395]:
def baseline_df(results):
    results = baseline(results)
    label = results[0].split(',')
    
    results_n = pd.DataFrame(results[1])
    results_n['Label'] = label
    results_n.set_index(['Label'], inplace = True)
    
    return(results_n)


In [2]:
result3 = baseline_df(results3)
display(result3)    #baseline accuracy for SF + these n reporting areas denoted by 'correct' row

# Baseline for various n closest reporting areas

In [453]:
#coloring text for readability
    #source: https://stackoverflow.com/questions/287871/print-in-terminal-with-colors

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    
print (bcolors.WARNING + "Test" + bcolors.ENDC)

Test


In [454]:
data_f, data_o = data_fo(data, 3)

['Oakland', 'San Rafael', 'Redwood City', 'San Francisco']
(1560, 21)
['Good' 'Moderate' 'Unhealthy for Sensitive Groups']
['Oakland', 'San Rafael', 'Redwood City']
(1080, 21)
['Moderate' 'Good' 'Unhealthy for Sensitive Groups' 'Unhealthy']


In [480]:
def baseline_n(datadf, n, state = 'CA', reparea = 'San Francisco'):    
    #filter relevant data
    
    print(bcolors.BOLD + str(n)+' closest reporting areas, Total values collected, Distinct AQI categories observed' + bcolors.ENDC)
    print(bcolors.BOLD + 'Forecasted values' + bcolors.ENDC)
    data_f = forecasted(datadf, n, state, reparea)
    print(bcolors.BOLD + 'Observed values' + bcolors.ENDC)
    data_o = observed(datadf, n, state, reparea)
    print('\n')
    
    #comparison of impt values
    print(bcolors.BOLD + 'Forecasted missing values' + bcolors.ENDC)
    data_f = compare(data_f)
    print('\n')
    print(bcolors.BOLD + 'Observed missing values' + bcolors.ENDC)
    data_o = compare(data_o)
    print('\n')
    
    #structure into dfs and concat
    data_f = pd.DataFrame(data_f)
    data_o = pd.DataFrame(data_o)
    dfs = {'For': data_f, 'Obs': data_o}
    keys = ['{}'.format(df) for df in list(dfs.keys())]
    results = pd.concat([data_f, data_o], axis = 1, keys = keys)
    #print(results['reporting area'].unique())
    
    #calc accuracy results
    print(bcolors.BOLD + 'Number of predictions' + bcolors.ENDC)
    results_n = baseline_df(results)
    
    #return results for given n
    return(results_n)

In [481]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [485]:
repareas = [1, 3, 5]

#prints results, no return
def n_results(repareas):
    for n in repareas:
        print(bcolors.OKBLUE + bcolors.BOLD + bcolors.UNDERLINE + 
                str('Baseline Accuracy for '+str(n)+' closest reporting areas').upper()
                 + bcolors.ENDC, '\n')
        results = baseline_n(data, n)
        display(results)
        print('\n\n\n\n\n\n')

In [488]:
n_results([1])

BASELINE ACCURACY FOR 1 CLOSEST REPORTING AREAS 

1 closest reporting areas, Total values collected, Distinct AQI categories observed
Forecasted values
['Oakland', 'San Francisco']
(780, 21)
['Good' 'Moderate' 'Unhealthy for Sensitive Groups']
Observed values
['Oakland', 'San Francisco']
(726, 21)
['Moderate' 'Good' 'Unhealthy for Sensitive Groups' 'Unhealthy']


Forecasted missing values
Number of Dates missing aqi in Oakland: 0
Number of Dates missing aqi in San Francisco: 0


Observed missing values
Number of Dates missing aqi in Oakland: 2
Number of Dates missing aqi in San Francisco: 12


Number of predictions
Oakland
San Francisco
390


,Oakland,San Francisco
Label,,
correct,0.5821,0.5897
incorrect,0.3667,0.3590
missing_all,0.0000,0.0000
missing_for,0.0000,0.0000
missing_obs,0.0513,0.0513


In [489]:
n_results(repareas)

BASELINE ACCURACY FOR 1 CLOSEST REPORTING AREAS 

1 closest reporting areas, Total values collected, Distinct AQI categories observed
Forecasted values
['Oakland', 'San Francisco']
(780, 21)
['Good' 'Moderate' 'Unhealthy for Sensitive Groups']
Observed values
['Oakland', 'San Francisco']
(726, 21)
['Moderate' 'Good' 'Unhealthy for Sensitive Groups' 'Unhealthy']


Forecasted missing values
Number of Dates missing aqi in Oakland: 0
Number of Dates missing aqi in San Francisco: 0


Observed missing values
Number of Dates missing aqi in Oakland: 2
Number of Dates missing aqi in San Francisco: 12


Number of predictions
Oakland
San Francisco
390


,Oakland,San Francisco
Label,,
correct,0.5821,0.5897
incorrect,0.3667,0.3590
missing_all,0.0000,0.0000
missing_for,0.0000,0.0000
missing_obs,0.0513,0.0513



















BASELINE ACCURACY FOR 3 CLOSEST REPORTING AREAS 

3 closest reporting areas, Total values collected, Distinct AQI categories observed
Forecasted values
['Oakland', 'San Rafael', 'Redwood City', 'San Francisco']
(1560, 21)
['Good' 'Moderate' 'Unhealthy for Sensitive Groups']
Observed values
['Oakland', 'San Rafael', 'Redwood City', 'San Francisco']
(1438, 21)
['Moderate' 'Good' 'Unhealthy for Sensitive Groups' 'Unhealthy']


Forecasted missing values
Number of Dates missing aqi in Oakland: 0
Number of Dates missing aqi in Redwood City: 0
Number of Dates missing aqi in San Francisco: 0
Number of Dates missing aqi in San Rafael: 0


Observed missing values
Number of Dates missing aqi in Oakland: 3
Number of Dates missing aqi in Redwood City: 21
Number of Dates missing aqi in San Francisco: 13
Number of Dates missing aqi in San Rafael: 9


Number of predictions
Oakland
Redwood City
San Francisco
San Rafael
390


,Oakland,Redwood City,San Francisco,San Rafael
Label,,,,
correct,0.5821,0.7282,0.5897,0.5436
incorrect,0.3692,0.2231,0.3615,0.4077
missing_all,0.0000,0.0000,0.0000,0.0000
missing_for,0.0000,0.0000,0.0000,0.0000
missing_obs,0.0487,0.0487,0.0487,0.0487



















BASELINE ACCURACY FOR 5 CLOSEST REPORTING AREAS 

5 closest reporting areas, Total values collected, Distinct AQI categories observed
Forecasted values
['Oakland', 'San Rafael', 'Redwood City', 'Fremont', 'Concord', 'San Francisco']
(2340, 21)
['Good' 'Moderate' 'Unhealthy for Sensitive Groups']
Observed values
['Oakland', 'San Rafael', 'Redwood City', 'Fremont', 'Concord', 'San Francisco']
(2170, 21)
['Moderate' 'Good' 'Unhealthy for Sensitive Groups' 'Unhealthy']


Forecasted missing values
Number of Dates missing aqi in Concord: 0
Number of Dates missing aqi in Fremont: 0
Number of Dates missing aqi in Oakland: 0
Number of Dates missing aqi in Redwood City: 0
Number of Dates missing aqi in San Francisco: 0
Number of Dates missing aqi in San Rafael: 0


Observed missing values
Number of Dates missing aqi in Concord: 10
Number of Dates missing aqi in Fremont: 0
Number of Dates missing aqi in Oakland: 3
Number of Dates missing aqi in Redwood City: 21
Number of Dates mi

,Concord,Fremont,Oakland,Redwood City,San Francisco,San Rafael
Label,,,,,,
correct,0.5949,0.7667,0.5821,0.7282,0.5897,0.5436
incorrect,0.3564,0.1846,0.3692,0.2231,0.3615,0.4077
missing_all,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
missing_for,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
missing_obs,0.0487,0.0487,0.0487,0.0487,0.0487,0.0487
